# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 27 2022 12:30PM,249525,19,8686745,Eywa Pharma Inc.,Released
1,Oct 27 2022 12:30PM,249525,19,8691191,Eywa Pharma Inc.,Released
2,Oct 27 2022 12:24PM,249531,10,0086122705,ISDIN Corporation,Released
3,Oct 27 2022 12:24PM,249531,10,0086122723,ISDIN Corporation,Released
4,Oct 27 2022 12:24PM,249531,10,0086122794,ISDIN Corporation,Released
5,Oct 27 2022 12:24PM,249531,10,0086122819,ISDIN Corporation,Released
6,Oct 27 2022 12:24PM,249531,10,0086122849,ISDIN Corporation,Released
7,Oct 27 2022 12:24PM,249531,10,0086122847,ISDIN Corporation,Released
8,Oct 27 2022 12:24PM,249531,10,0086122851,ISDIN Corporation,Released
9,Oct 27 2022 12:24PM,249531,10,0086122852,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,249527,Released,3
26,249528,Released,4
27,249529,Released,3
28,249530,Released,39
29,249531,Released,12


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
249527,NaN,3.0
249528,NaN,4.0
249529,NaN,3.0
249530,NaN,39.0
249531,NaN,12.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
248349,0.0,1.0
249451,1.0,0.0
249472,25.0,16.0
249479,1.0,0.0
249485,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
248349,0,1
249451,1,0
249472,25,16
249479,1,0
249485,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,248349,0,1
1,249451,1,0
2,249472,25,16
3,249479,1,0
4,249485,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,248349,,1
1,249451,1,
2,249472,25,16
3,249479,1,
4,249485,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 27 2022 12:30PM,249525,19,Eywa Pharma Inc.
2,Oct 27 2022 12:24PM,249531,10,ISDIN Corporation
14,Oct 27 2022 12:21PM,249528,10,"Methapharm, Inc."
18,Oct 27 2022 12:19PM,249530,15,"Person & Covey, Inc."
57,Oct 27 2022 12:16PM,249529,10,"Methapharm, Inc."
60,Oct 27 2022 12:15PM,249527,10,Emerginnova
63,Oct 27 2022 12:15PM,249526,21,"NBTY Global, Inc."
64,Oct 27 2022 12:14PM,249524,21,"NBTY Global, Inc."
65,Oct 27 2022 12:13PM,249523,21,"NBTY Global, Inc."
66,Oct 27 2022 12:08PM,249521,16,TASA USA Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Oct 27 2022 12:30PM,249525,19,Eywa Pharma Inc.,,2
1,Oct 27 2022 12:24PM,249531,10,ISDIN Corporation,,12
2,Oct 27 2022 12:21PM,249528,10,"Methapharm, Inc.",,4
3,Oct 27 2022 12:19PM,249530,15,"Person & Covey, Inc.",,39
4,Oct 27 2022 12:16PM,249529,10,"Methapharm, Inc.",,3
5,Oct 27 2022 12:15PM,249527,10,Emerginnova,,3
6,Oct 27 2022 12:15PM,249526,21,"NBTY Global, Inc.",,1
7,Oct 27 2022 12:14PM,249524,21,"NBTY Global, Inc.",,1
8,Oct 27 2022 12:13PM,249523,21,"NBTY Global, Inc.",,1
9,Oct 27 2022 12:08PM,249521,16,TASA USA Inc.,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 27 2022 12:30PM,249525,19,Eywa Pharma Inc.,2,
1,Oct 27 2022 12:24PM,249531,10,ISDIN Corporation,12,
2,Oct 27 2022 12:21PM,249528,10,"Methapharm, Inc.",4,
3,Oct 27 2022 12:19PM,249530,15,"Person & Covey, Inc.",39,
4,Oct 27 2022 12:16PM,249529,10,"Methapharm, Inc.",3,
5,Oct 27 2022 12:15PM,249527,10,Emerginnova,3,
6,Oct 27 2022 12:15PM,249526,21,"NBTY Global, Inc.",1,
7,Oct 27 2022 12:14PM,249524,21,"NBTY Global, Inc.",1,
8,Oct 27 2022 12:13PM,249523,21,"NBTY Global, Inc.",1,
9,Oct 27 2022 12:08PM,249521,16,TASA USA Inc.,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 27 2022 12:30PM,249525,19,Eywa Pharma Inc.,2,
1,Oct 27 2022 12:24PM,249531,10,ISDIN Corporation,12,
2,Oct 27 2022 12:21PM,249528,10,"Methapharm, Inc.",4,
3,Oct 27 2022 12:19PM,249530,15,"Person & Covey, Inc.",39,
4,Oct 27 2022 12:16PM,249529,10,"Methapharm, Inc.",3,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Oct 27 2022 12:30PM,249525,19,Eywa Pharma Inc.,2.0,NaN
1,Oct 27 2022 12:24PM,249531,10,ISDIN Corporation,12.0,NaN
2,Oct 27 2022 12:21PM,249528,10,"Methapharm, Inc.",4.0,NaN
3,Oct 27 2022 12:19PM,249530,15,"Person & Covey, Inc.",39.0,NaN
4,Oct 27 2022 12:16PM,249529,10,"Methapharm, Inc.",3.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 27 2022 12:30PM,249525,19,Eywa Pharma Inc.,2.0,0.0
1,Oct 27 2022 12:24PM,249531,10,ISDIN Corporation,12.0,0.0
2,Oct 27 2022 12:21PM,249528,10,"Methapharm, Inc.",4.0,0.0
3,Oct 27 2022 12:19PM,249530,15,"Person & Covey, Inc.",39.0,0.0
4,Oct 27 2022 12:16PM,249529,10,"Methapharm, Inc.",3.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1497142,24.0,0.0
12,499007,1.0,1.0
15,1497054,88.0,26.0
16,998039,3.0,1.0
19,499034,3.0,0.0
20,498979,1.0,1.0
21,1495875,5.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1497142,24.0,0.0
1,12,499007,1.0,1.0
2,15,1497054,88.0,26.0
3,16,998039,3.0,1.0
4,19,499034,3.0,0.0
5,20,498979,1.0,1.0
6,21,1495875,5.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,24.0,0.0
1,12,1.0,1.0
2,15,88.0,26.0
3,16,3.0,1.0
4,19,3.0,0.0
5,20,1.0,1.0
6,21,5.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,24.0
1,12,Released,1.0
2,15,Released,88.0
3,16,Released,3.0
4,19,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Executing,0.0,1.0,26.0,1.0,0.0,1.0,1.0
Released,24.0,1.0,88.0,3.0,3.0,1.0,5.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Executing,0.0,1.0,26.0,1.0,0.0,1.0,1.0
1,Released,24.0,1.0,88.0,3.0,3.0,1.0,5.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Executing,0.0,1.0,26.0,1.0,0.0,1.0,1.0
1,Released,24.0,1.0,88.0,3.0,3.0,1.0,5.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()